# Joining DataFrames

Joining data from multiple sources is a fundamental operation in data analysis. While R's `dplyr::*_join()` functions provide a consistent interface for different types of joins, pandas offers `merge()`, `join()`, and `concat()` with various options. This chapter will show you how to achieve tidyverse-style joins in pandas.

## Best Practices Summary

Quick reference for join patterns:

| Task | R (dplyr) | Pandas |
|------|-----------|--------|
| Inner join | `inner_join(df1, df2, by = "key")` | `pd.merge(df1, df2, on='key')` |
| Left join | `left_join(df1, df2, by = "key")` | `pd.merge(df1, df2, on='key', how='left')` |
| Right join | `right_join(df1, df2, by = "key")` | `pd.merge(df1, df2, on='key', how='right')` |
| Full join | `full_join(df1, df2, by = "key")` | `pd.merge(df1, df2, on='key', how='outer')` |
| Multiple keys | `left_join(df1, df2, by = c("key1", "key2"))` | `pd.merge(df1, df2, on=['key1', 'key2'], how='left')` |
| Different names | `left_join(df1, df2, by = c("id" = "ID"))` | `pd.merge(df1, df2, left_on='id', right_on='ID', how='left')` |
| Semi join | `semi_join(df1, df2, by = "key")` | `df1[df1['key'].isin(df2['key'])]` |
| Anti join | `anti_join(df1, df2, by = "key")` | `df1[~df1['key'].isin(df2['key'])]` |
| Cross join | `cross_join(df1, df2)` | Add temporary key and merge |

## Tips for Tidyverse Users

1. **Default is inner join**: Unlike some R functions, pandas defaults to inner join, not left join.

2. **Use `indicator=True`**: The `indicator` parameter adds a column showing which DataFrame(s) each row came from.

3. **Handle duplicates**: Check for duplicate keys before joining to avoid unexpected row multiplication.

4. **Consider indexes**: For repeated joins on the same column, setting it as index can improve performance.

5. **Chain joins carefully**: When chaining multiple joins, be mindful of the order and join types:
   ```python
   (df1
    .merge(df2, on='key1', how='left')
    .merge(df3, on='key2', how='left')
    .merge(df4, on='key3', how='inner'))  # Final inner join filters results
   ```

Joining DataFrames in pandas is powerful and flexible. While the syntax differs from dplyr's join functions, pandas `merge()` offers fine-grained control over join operations, and understanding its parameters will help you handle any joining scenario efficiently.

## Basic Joins

The fundamental join operations in pandas:

In [1]:
import pandas as pd
import numpy as np

# Create sample DataFrames
employees = pd.DataFrame({
    'employee_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'department_id': [101, 102, 101, 103, 102],
    'salary': [70000, 85000, 65000, 72000, 90000]
})

departments = pd.DataFrame({
    'department_id': [101, 102, 103, 104],
    'department_name': ['Sales', 'IT', 'HR', 'Marketing'],
    'location': ['New York', 'San Francisco', 'Chicago', 'Boston']
})

print("Employees:")
employees

Employees:


,employee_id,name,department_id,salary
0,1,Alice,101,70000
1,2,Bob,102,85000
2,3,Charlie,101,65000
3,4,David,103,72000
4,5,Eve,102,90000


In [2]:
print("Departments:")
departments

Departments:


,department_id,department_name,location
0,101,Sales,New York
1,102,IT,San Francisco
2,103,HR,Chicago
3,104,Marketing,Boston


In [3]:
# Inner join (default)
# R: inner_join(employees, departments, by = "department_id")
pd.merge(employees, departments, on='department_id')

,employee_id,name,department_id,salary,department_name,location
0,1,Alice,101,70000,Sales,New York
1,2,Bob,102,85000,IT,San Francisco
2,3,Charlie,101,65000,Sales,New York
3,4,David,103,72000,HR,Chicago
4,5,Eve,102,90000,IT,San Francisco


In [4]:
# Left join
# R: left_join(employees, departments, by = "department_id")
pd.merge(employees, departments, on='department_id', how='left')

,employee_id,name,department_id,salary,department_name,location
0,1,Alice,101,70000,Sales,New York
1,2,Bob,102,85000,IT,San Francisco
2,3,Charlie,101,65000,Sales,New York
3,4,David,103,72000,HR,Chicago
4,5,Eve,102,90000,IT,San Francisco


In [5]:
# Right join
# R: right_join(employees, departments, by = "department_id")
pd.merge(employees, departments, on='department_id', how='right')

,employee_id,name,department_id,salary,department_name,location
0,1.0,Alice,101,70000.0,Sales,New York
1,3.0,Charlie,101,65000.0,Sales,New York
2,2.0,Bob,102,85000.0,IT,San Francisco
3,5.0,Eve,102,90000.0,IT,San Francisco
4,4.0,David,103,72000.0,HR,Chicago
5,NaN,NaN,104,NaN,Marketing,Boston


In [6]:
# Full outer join
# R: full_join(employees, departments, by = "department_id")
pd.merge(employees, departments, on='department_id', how='outer')

,employee_id,name,department_id,salary,department_name,location
0,1.0,Alice,101,70000.0,Sales,New York
1,3.0,Charlie,101,65000.0,Sales,New York
2,2.0,Bob,102,85000.0,IT,San Francisco
3,5.0,Eve,102,90000.0,IT,San Francisco
4,4.0,David,103,72000.0,HR,Chicago
5,NaN,NaN,104,NaN,Marketing,Boston


## Joining on Multiple Columns

Joining on multiple keys:

In [7]:
# Create DataFrames with multiple join keys
sales = pd.DataFrame({
    'year': [2023, 2023, 2023, 2024, 2024, 2024],
    'quarter': ['Q1', 'Q2', 'Q3', 'Q1', 'Q2', 'Q3'],
    'region': ['East', 'East', 'East', 'East', 'East', 'East'],
    'sales_amount': [100000, 120000, 115000, 130000, 125000, 140000]
})

targets = pd.DataFrame({
    'year': [2023, 2023, 2024, 2024],
    'quarter': ['Q1', 'Q2', 'Q1', 'Q2'],
    'region': ['East', 'East', 'East', 'East'],
    'target_amount': [95000, 110000, 120000, 130000]
})

# Join on multiple columns
# R: left_join(sales, targets, by = c("year", "quarter", "region"))
pd.merge(sales, targets, on=['year', 'quarter', 'region'], how='left')

,year,quarter,region,sales_amount,target_amount
0,2023,Q1,East,100000,95000.0
1,2023,Q2,East,120000,110000.0
2,2023,Q3,East,115000,NaN
3,2024,Q1,East,130000,120000.0
4,2024,Q2,East,125000,130000.0
5,2024,Q3,East,140000,NaN


## Joining with Different Column Names

When column names don't match:

In [8]:
# Create DataFrames with different column names
employees2 = pd.DataFrame({
    'emp_id': [1, 2, 3, 4, 5],
    'emp_name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'dept_code': [101, 102, 101, 103, 102]
})

departments2 = pd.DataFrame({
    'department_id': [101, 102, 103, 104],
    'department': ['Sales', 'IT', 'HR', 'Marketing']
})

# Join with different column names
# R: left_join(employees2, departments2, by = c("dept_code" = "department_id"))
pd.merge(employees2, departments2, 
         left_on='dept_code', 
         right_on='department_id', 
         how='left')

,emp_id,emp_name,dept_code,department_id,department
0,1,Alice,101,101,Sales
1,2,Bob,102,102,IT
2,3,Charlie,101,101,Sales
3,4,David,103,103,HR
4,5,Eve,102,102,IT


In [9]:
# Drop duplicate column after merge
result = pd.merge(employees2, departments2, 
                  left_on='dept_code', 
                  right_on='department_id', 
                  how='left')
result.drop(columns='department_id')  # Keep only one of the join columns

,emp_id,emp_name,dept_code,department
0,1,Alice,101,Sales
1,2,Bob,102,IT
2,3,Charlie,101,Sales
3,4,David,103,HR
4,5,Eve,102,IT


## Handling Duplicate Column Names

Managing columns with same names:

In [10]:
# Create DataFrames with overlapping column names
df1 = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['Alice', 'Bob', 'Charlie'],
    'value': [100, 200, 300],
    'date': pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03'])
})

df2 = pd.DataFrame({
    'id': [1, 2, 3],
    'value': [10, 20, 30],
    'status': ['active', 'active', 'inactive']
})

# Merge with suffix for duplicate columns
# R: left_join(df1, df2, by = "id", suffix = c("_df1", "_df2"))
pd.merge(df1, df2, on='id', suffixes=('_df1', '_df2'))

,id,name,value_df1,date,value_df2,status
0,1,Alice,100,2024-01-01,10,active
1,2,Bob,200,2024-01-02,20,active
2,3,Charlie,300,2024-01-03,30,inactive


## Semi and Anti Joins

Filtering joins (semi and anti joins):

In [11]:
# Create sample data
customers = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5],
    'name': ['Customer A', 'Customer B', 'Customer C', 'Customer D', 'Customer E'],
    'city': ['NYC', 'LA', 'Chicago', 'Houston', 'Phoenix']
})

orders = pd.DataFrame({
    'order_id': [101, 102, 103, 104],
    'customer_id': [1, 2, 1, 3],
    'amount': [1000, 1500, 800, 2000]
})

# Semi join - keep only customers with orders
# R: semi_join(customers, orders, by = "customer_id")
customers_with_orders = customers[customers['customer_id'].isin(orders['customer_id'])]
customers_with_orders

,customer_id,name,city
0,1,Customer A,NYC
1,2,Customer B,LA
2,3,Customer C,Chicago


In [12]:
# Anti join - keep only customers without orders
# R: anti_join(customers, orders, by = "customer_id")
customers_without_orders = customers[~customers['customer_id'].isin(orders['customer_id'])]
customers_without_orders

,customer_id,name,city
3,4,Customer D,Houston
4,5,Customer E,Phoenix


In [13]:
# Alternative semi-join using merge
semi_join_result = pd.merge(customers, orders[['customer_id']].drop_duplicates(), 
                           on='customer_id', 
                           how='inner')
semi_join_result

,customer_id,name,city
0,1,Customer A,NYC
1,2,Customer B,LA
2,3,Customer C,Chicago


## Cross Joins

Creating cartesian products:

In [14]:
# Create small DataFrames for cross join
sizes = pd.DataFrame({'size': ['S', 'M', 'L']})
colors = pd.DataFrame({'color': ['Red', 'Blue', 'Green']})

# Cross join - all combinations
# R: cross_join(sizes, colors) or expand_grid(size = c("S", "M", "L"), color = c("Red", "Blue", "Green"))
sizes['key'] = 1
colors['key'] = 1
cross_joined = pd.merge(sizes, colors, on='key').drop(columns='key')
cross_joined

,size,color
0,S,Red
1,S,Blue
2,S,Green
3,M,Red
4,M,Blue
5,M,Green
6,L,Red
7,L,Blue
8,L,Green


In [15]:
# Alternative using itertools
import itertools
cross_join_alt = pd.DataFrame(
    list(itertools.product(sizes['size'], colors['color'])),
    columns=['size', 'color']
)
cross_join_alt

,size,color
0,S,Red
1,S,Blue
2,S,Green
3,M,Red
4,M,Blue
5,M,Green
6,L,Red
7,L,Blue
8,L,Green


## Joining with Index

Using index for joins:

In [16]:
# Create DataFrames with meaningful index
products_indexed = pd.DataFrame({
    'product_name': ['Widget', 'Gadget', 'Doohickey'],
    'price': [19.99, 29.99, 39.99]
}, index=['P001', 'P002', 'P003'])

inventory_indexed = pd.DataFrame({
    'quantity': [100, 50, 75],
    'warehouse': ['A', 'A', 'B']
}, index=['P001', 'P002', 'P003'])

# Join on index
# No direct R equivalent - would need to convert rownames to column
products_indexed.join(inventory_indexed)

,product_name,price,quantity,warehouse
P001,Widget,19.99,100,A
P002,Gadget,29.99,50,A
P003,Doohickey,39.99,75,B


In [17]:
# Join DataFrame with index to DataFrame with column
sales_by_product = pd.DataFrame({
    'product_id': ['P001', 'P002', 'P001', 'P003'],
    'units_sold': [10, 5, 15, 8],
    'date': pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04'])
})

# Merge indexed DataFrame with regular DataFrame
pd.merge(sales_by_product, products_indexed, 
         left_on='product_id', 
         right_index=True)

,product_id,units_sold,date,product_name,price
0,P001,10,2024-01-01,Widget,19.99
1,P002,5,2024-01-02,Gadget,29.99
2,P001,15,2024-01-03,Widget,19.99
3,P003,8,2024-01-04,Doohickey,39.99


## Multiple DataFrames Join

Joining more than two DataFrames:

In [18]:
# Create multiple related DataFrames
employees = pd.DataFrame({
    'emp_id': [1, 2, 3, 4],
    'name': ['Alice', 'Bob', 'Charlie', 'David'],
    'dept_id': [10, 20, 10, 30],
    'manager_id': [3, 3, 4, 4]
})

departments = pd.DataFrame({
    'dept_id': [10, 20, 30],
    'dept_name': ['Sales', 'IT', 'HR'],
    'budget': [100000, 150000, 80000]
})

managers = pd.DataFrame({
    'manager_id': [3, 4],
    'manager_name': ['Charlie', 'David'],
    'level': ['Senior', 'Executive']
})

# Chain multiple joins
# R: employees %>% 
#     left_join(departments, by = "dept_id") %>%
#     left_join(managers, by = "manager_id")
result = (employees
    .merge(departments, on='dept_id', how='left')
    .merge(managers, on='manager_id', how='left')
)
result

,emp_id,name,dept_id,manager_id,dept_name,budget,manager_name,level
0,1,Alice,10,3,Sales,100000,Charlie,Senior
1,2,Bob,20,3,IT,150000,Charlie,Senior
2,3,Charlie,10,4,Sales,100000,David,Executive
3,4,David,30,4,HR,80000,David,Executive


In [19]:
# Using reduce for multiple joins
from functools import reduce

# List of DataFrames to join
dfs = [employees, departments, managers]

# Define join keys for each pair
join_keys = ['dept_id', 'manager_id']

# Perform sequential joins
result_reduce = reduce(
    lambda left, right: pd.merge(left, right[1], 
                                on=join_keys[right[0]], 
                                how='left'),
    enumerate(dfs[1:]), 
    dfs[0]
)

## Inequality Joins

Joining on conditions other than equality:

In [20]:
# Create DataFrames for inequality join
events = pd.DataFrame({
    'event_id': [1, 2, 3, 4],
    'start_time': pd.to_datetime(['2024-01-01 09:00', '2024-01-01 14:00', 
                                  '2024-01-02 10:00', '2024-01-02 15:00']),
    'end_time': pd.to_datetime(['2024-01-01 11:00', '2024-01-01 16:00', 
                               '2024-01-02 12:00', '2024-01-02 17:00']),
    'event_name': ['Meeting A', 'Meeting B', 'Meeting C', 'Meeting D']
})

timestamps = pd.DataFrame({
    'check_time': pd.to_datetime(['2024-01-01 10:00', '2024-01-01 15:00', 
                                  '2024-01-02 11:00', '2024-01-02 18:00']),
    'person': ['Alice', 'Bob', 'Charlie', 'David']
})

# Find which events each person could have attended
# This requires a cross join followed by filtering
events['key'] = 1
timestamps['key'] = 1
cross = pd.merge(events, timestamps, on='key').drop(columns='key')

# Filter for overlapping times
result = cross[
    (cross['check_time'] >= cross['start_time']) & 
    (cross['check_time'] <= cross['end_time'])
][['person', 'event_name', 'check_time']]
result

,person,event_name,check_time
0,Alice,Meeting A,2024-01-01 10:00:00
5,Bob,Meeting B,2024-01-01 15:00:00
10,Charlie,Meeting C,2024-01-02 11:00:00


## Joining with Different Data Types

Handling type mismatches in joins:

In [21]:
# Create DataFrames with different types
df_int = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'value': [100, 200, 300, 400]
})

df_str = pd.DataFrame({
    'id': ['1', '2', '3', '5'],  # String type IDs
    'category': ['A', 'B', 'C', 'D']
})

# This will not match due to type difference
try:
    result = pd.merge(df_int, df_str, on='id')
    print(f"Rows matched: {len(result)}")
except:
    print("Join failed due to type mismatch")

# Convert types before joining
df_str['id'] = df_str['id'].astype(int)
pd.merge(df_int, df_str, on='id', how='outer')

Join failed due to type mismatch


,id,value,category
0,1,100.0,A
1,2,200.0,B
2,3,300.0,C
3,4,400.0,NaN
4,5,NaN,D


## Performance Optimization

Efficient joining strategies:

In [22]:
# Create large DataFrames for performance testing
np.random.seed(42)
n = 100000
large_df1 = pd.DataFrame({
    'key': np.random.randint(0, 10000, n),
    'value1': np.random.randn(n)
})

large_df2 = pd.DataFrame({
    'key': np.random.randint(0, 10000, n),
    'value2': np.random.randn(n)
})

import time

# Method 1: Regular merge
start = time.time()
result1 = pd.merge(large_df1, large_df2, on='key')
print(f"Regular merge: {time.time() - start:.4f} seconds")

# Method 2: Sort then merge (can be faster for large data)
start = time.time()
large_df1_sorted = large_df1.sort_values('key')
large_df2_sorted = large_df2.sort_values('key')
result2 = pd.merge(large_df1_sorted, large_df2_sorted, on='key')
print(f"Sort then merge: {time.time() - start:.4f} seconds")

# Method 3: Set index then join (efficient for multiple joins)
start = time.time()
df1_indexed = large_df1.set_index('key')
df2_indexed = large_df2.set_index('key')
result3 = df1_indexed.join(df2_indexed, how='inner')
print(f"Index join: {time.time() - start:.4f} seconds")

Regular merge: 0.0589 seconds
Sort then merge: 0.0454 seconds
Index join: 0.0350 seconds


## Validation and Debugging Joins

Checking join results:

In [23]:
# Create DataFrames with potential issues
main_df = pd.DataFrame({
    'id': [1, 2, 2, 3, 4],  # Duplicate ID
    'name': ['A', 'B', 'B2', 'C', 'D']
})

lookup_df = pd.DataFrame({
    'id': [1, 2, 3, 3, 5],  # Duplicate ID and missing ID 4
    'value': [100, 200, 300, 350, 500]
})

# Check for duplicates before joining
print("Duplicates in main_df:", main_df['id'].duplicated().sum())
print("Duplicates in lookup_df:", lookup_df['id'].duplicated().sum())

# Perform join and validate
result = pd.merge(main_df, lookup_df, on='id', how='left', indicator=True)
result

Duplicates in main_df: 1
Duplicates in lookup_df: 1


,id,name,value,_merge
0,1,A,100.0,both
1,2,B,200.0,both
2,2,B2,200.0,both
3,3,C,300.0,both
4,3,C,350.0,both
5,4,D,NaN,left_only


In [24]:
# Analyze join results
print("\nJoin result summary:")
print(result['_merge'].value_counts())


Join result summary:
_merge
both          5
left_only     1
right_only    0
Name: count, dtype: int64


## Creating Tidyverse-Style Join Functions

Make joining more dplyr-like:

In [25]:
def inner_join(df1, df2, by=None, on=None):
    """Mimics dplyr's inner_join"""
    key = by or on
    return pd.merge(df1, df2, on=key, how='inner')

def left_join(df1, df2, by=None, on=None):
    """Mimics dplyr's left_join"""
    key = by or on
    return pd.merge(df1, df2, on=key, how='left')

def right_join(df1, df2, by=None, on=None):
    """Mimics dplyr's right_join"""
    key = by or on
    return pd.merge(df1, df2, on=key, how='right')

def full_join(df1, df2, by=None, on=None):
    """Mimics dplyr's full_join"""
    key = by or on
    return pd.merge(df1, df2, on=key, how='outer')

def semi_join(df1, df2, by=None, on=None):
    """Mimics dplyr's semi_join"""
    key = by or on
    if isinstance(key, list):
        return df1[df1[key].apply(tuple, axis=1).isin(df2[key].apply(tuple, axis=1))]
    return df1[df1[key].isin(df2[key])]

def anti_join(df1, df2, by=None, on=None):
    """Mimics dplyr's anti_join"""
    key = by or on
    if isinstance(key, list):
        return df1[~df1[key].apply(tuple, axis=1).isin(df2[key].apply(tuple, axis=1))]
    return df1[~df1[key].isin(df2[key])]

# Usage examples
left_join(employees, departments, by='department_id')

KeyError: 'department_id'